# Contando M&Ms en Python

1. Importamos las librerias necesarias y establecemos el nombre del fichero con los datos

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import count, avg, col, max, min, sum

mnm_file = "mnm_dataset.csv"

2. Construimos el SparkSession, si no existe se crea una instancia

In [2]:
spark = (SparkSession
    .builder
    .appName("PythonMnMCount")
    .getOrCreate())

3. Leemos el fichero csv y lo convertimos a un Spark Data Frame. Para ello, le inferimos el esquema desde el fichero y especificamos que el archivo tiene un header con los nombres de las columnas.

In [3]:
mnm_df = (spark.read.format("csv")
    .option("header", "true")
    .option("inferSchema", "true")
    .load(mnm_file))
mnm_df.persist()

DataFrame[State: string, Color: string, Count: int]

4. Usamos la API de DataFrames:
    - Selecionamos los campos "State", "Color" y "Count"
    - Agrupamos por "State" y por "Color"
    - Agregamos los "Count" de cada grupo en una columna "Total"
    - Ordenamos por el "Total" en orden descendente

In [4]:
count_mnm_df = (mnm_df
    .select("State", "Color", "Count")
    .groupBy("State", "Color")
    .agg(count("Count").alias("Total"))
    .orderBy("Total", ascending=False))
count_mnm_df.show(n=60, truncate=False)

+-----+------+-----+
|State|Color |Total|
+-----+------+-----+
|CA   |Yellow|1807 |
|WA   |Green |1779 |
|OR   |Orange|1743 |
|TX   |Green |1737 |
|TX   |Red   |1725 |
|CA   |Green |1723 |
|CO   |Yellow|1721 |
|CA   |Brown |1718 |
|CO   |Green |1713 |
|NV   |Orange|1712 |
|TX   |Yellow|1703 |
|NV   |Green |1698 |
|AZ   |Brown |1698 |
|WY   |Green |1695 |
|CO   |Blue  |1695 |
|NM   |Red   |1690 |
|AZ   |Orange|1689 |
|NM   |Yellow|1688 |
|NM   |Brown |1687 |
|UT   |Orange|1684 |
|NM   |Green |1682 |
|UT   |Red   |1680 |
|AZ   |Green |1676 |
|NV   |Yellow|1675 |
|NV   |Blue  |1673 |
|WA   |Red   |1671 |
|WY   |Red   |1670 |
|WA   |Brown |1669 |
|NM   |Orange|1665 |
|WY   |Blue  |1664 |
|WA   |Yellow|1663 |
|WA   |Orange|1658 |
|CA   |Orange|1657 |
|NV   |Brown |1657 |
|CO   |Brown |1656 |
|CA   |Red   |1656 |
|UT   |Blue  |1655 |
|AZ   |Yellow|1654 |
|TX   |Orange|1652 |
|AZ   |Red   |1648 |
|OR   |Blue  |1646 |
|UT   |Yellow|1645 |
|OR   |Red   |1645 |
|CO   |Orange|1642 |
|TX   |Brown 

- Ejemplo usando *avg*, calculamos la media de cada color entre todos los estados:

In [5]:
avg_mnm_color_df = (mnm_df
    .select("State", "Color", "Count")
    .groupBy("State", "Color")
    .agg(count("Count").alias("Total"))
    .groupBy("Color")
    .agg(avg("Total").alias("Average"))
    .orderBy("Average", ascending=True)
)
avg_mnm_color_df.show()

+------+-------+
| Color|Average|
+------+-------+
|  Blue| 1644.9|
| Brown| 1651.0|
|   Red| 1661.9|
|Orange| 1669.7|
|Yellow| 1679.6|
| Green| 1692.8|
+------+-------+



- Ejemplo usando la cláusula *where*, filtramos los estados que empiezan por vocal:

In [6]:
vowel_states_df = (mnm_df
    .select("State")
    .distinct()
    .where(col("State").rlike("^[AEIOU]"))
)
vowel_states_df.show()

+-----+
|State|
+-----+
|   AZ|
|   OR|
|   UT|
+-----+



- Ejemplo usando varias funciones de agregación. Se calcula el mínimo, máximo y media de votos de cada color entre todos los estados y la suma total de votos de cada color.

In [7]:
stats_mnm_color_df = (mnm_df
    .select("State", "Color", "Count")
    .groupBy("Color", "State")
    .agg(count("Count").alias("Total"))
    .groupBy("Color")
    .agg(min("Total").alias("Min votes"), max("Total").alias("Max votes"), avg("Total").alias("Average votes"), sum("Total").alias("Sum of votes"))
)
stats_mnm_color_df.show()

+------+---------+---------+-------------+------------+
| Color|Min votes|Max votes|Average votes|Sum of votes|
+------+---------+---------+-------------+------------+
|Orange|     1595|     1743|       1669.7|       16697|
| Green|     1591|     1779|       1692.8|       16928|
|  Blue|     1603|     1695|       1644.9|       16449|
| Brown|     1532|     1718|       1651.0|       16510|
|Yellow|     1614|     1807|       1679.6|       16796|
|   Red|     1610|     1725|       1661.9|       16619|
+------+---------+---------+-------------+------------+



- También podemos guardar nuestros dataframes como una vista temporal de SQL:

In [8]:
stats_mnm_color_df.createOrReplaceTempView("stats_color")
spark.table("stats_color").show()

+------+---------+---------+-------------+------------+
| Color|Min votes|Max votes|Average votes|Sum of votes|
+------+---------+---------+-------------+------------+
|Orange|     1595|     1743|       1669.7|       16697|
| Green|     1591|     1779|       1692.8|       16928|
|  Blue|     1603|     1695|       1644.9|       16449|
| Brown|     1532|     1718|       1651.0|       16510|
|Yellow|     1614|     1807|       1679.6|       16796|
|   Red|     1610|     1725|       1661.9|       16619|
+------+---------+---------+-------------+------------+



5. Detenemos el SparkSession

In [9]:
spark.stop()